In [2]:
:sccache 1
// Skip this or run `cargo install sccache`

sccache: true


In [3]:
:dep z3tracer
// Or use `:dep z3tracer = { path = ".." }` for the local version.

In [4]:
:dep plotters = { default_features = false, features = ["evcxr", "line_series"] }

In [5]:
:efmt {:?}

Error format: {:?} (errors must implement std::fmt::Debug)


In [6]:
:opt 3

Optimization: 3


In [7]:
use z3tracer::{syntax::*, model::*};
use std::collections::*;
use std::str::FromStr;
use plotters::prelude::*;
use plotters::evcxr::SVGWrapper;

In [8]:
:last_error_json
// BUG WORKAROUND: In case of mysteriously truncated error 'consider importing this struct', run the command above again
// to print a detailed error and find the import that is actually missing.
//
// See also :help for more useful commands

Error: 

In [9]:
/// Helper function to process files.
fn process_file(path: &str) -> std::io::Result<Model> {
    let file = std::io::BufReader::new(std::fs::File::open(path)?);
    // Inject non-default configurations here with Model::new(config).
    let mut model = Model::default();
    if let Err(le) = model.process(Some(path.to_string()), file) {
        println!("Error at {:?}: {:?}", le.position, le.error);
    }
    Ok(model)
}

/// Helper trait to print es by their id.
trait ModelExt {
    fn id2s(&self, id: &Ident) -> String;    
}

impl ModelExt for Model {
    fn id2s(&self, id: &Ident) -> String {
        self.id_to_sexp(&BTreeMap::new(), id).unwrap()
    }        
}

In [10]:
// TODO: remove after Rust issue 59278 is closed.
struct IntoIterSorted<T> {
    inner: BinaryHeap<T>,
}

impl<T> From<BinaryHeap<T>> for IntoIterSorted<T> {
    fn from(inner: BinaryHeap<T>) -> Self {
        Self { inner }
    }
}

impl<T: Ord> Iterator for IntoIterSorted<T> {
    type Item = T;

    fn next(&mut self) -> Option<T> {
        self.inner.pop()
    }

    fn size_hint(&self) -> (usize, Option<usize>) {
        let exact = self.inner.len();
        (exact, Some(exact))
    }
}

In [11]:
// Process some input file obtained with `z3 trace=true proof=true ..`
let model = process_file("../tests/data/file1.log")?;

In [12]:
// Maximum timestamp (i.e. currently, the number of lines in the Z3 log)
let max_ts = model.processed_logs();
max_ts

443166

In [13]:
// Compute top instantiated terms.
let mut top = model.most_instantiated_terms();
top.len()

66

In [14]:
// Peek at the 20th top elements
IntoIterSorted::from(top.clone()).take(20).collect::<Vec<_>>()

[(7903, basic#), (2955, datatype#85), (2955, datatype#82), (1863, datatype#), (1114, arith#), (769, datatype#59), (331, #4429!9), (304, #4328), (232, datatype#47), (224, #20151!10), (126, #20468!5), (120, #4351!1), (119, #23065!1), (119, #4347!1), (115, #4337), (100, #23092!1), (99, #23078!2), (97, datatype#53), (93, datatype#143), (93, datatype#140)]

In [15]:
// Compute top instantiated terms and retrieve the "timestamps" at which instantiations occur for each of the top terms.
let instantiation_times = IntoIterSorted::from(model.most_instantiated_terms()).map(|(_count, id)| {
    let mut timestamps = model.term_data(&id).unwrap().instantiations.iter().flat_map(|key| {
        let qi = model.instantiations().get(key).unwrap();
        qi.data.iter().map(|d| model.term_data(d.term.as_ref().unwrap()).unwrap().timestamp)
    }).collect::<Vec<_>>();
    timestamps.sort();
    let name = match model.term(&id).unwrap() { Term::Quant { name, .. } | Term::Builtin { name: Some(name) } => name, _ => "??" };
    (name.to_string(), timestamps)
}).collect::<Vec<_>>();

In [16]:
fn plot_instantiations(times: &[(String, Vec<usize>)], title: &str, top_n: usize) -> SVGWrapper {
    let max_ts = times.iter().map(|(_, v)| v.last().cloned().unwrap_or(0)).max().unwrap_or(1);
    let max_count = times[0].1.len();

    evcxr_figure((1000, 800), |root| {
        root.fill(&WHITE);
        let mut chart = ChartBuilder::on(&root)
            .caption(title, ("Arial", 30).into_font())
            .margin(10)
            .x_label_area_size(40)
            .y_label_area_size(50)
            .build_cartesian_2d(0..max_ts, 0..max_count)?;

        chart.configure_mesh().y_desc("Cumulative number of instantiations").x_desc("Time (line number)").draw()?;

        for (j, (name, values)) in times.iter().take(top_n).enumerate() {
            let color : PaletteColor<Palette9999> = PaletteColor::pick(j);
            chart
            .draw_series(
                LineSeries::new(values.iter().enumerate().map(|(i, c)| (*c, i)), &color)
            ).unwrap()
            .label(name)
            .legend(move |(x, y)| PathElement::new(vec![(x, y), (x + 20, y)], color.filled()));
        }

        chart.configure_series_labels()
            .background_style(&WHITE.mix(0.8))
            .border_style(&BLACK)
            .position(SeriesLabelPosition::UpperLeft)
            .draw()?;
        Ok(())
    })
}

In [17]:
plot_instantiations(&instantiation_times, "Top 10 Quantifiers Instantiations", 10)

Top 10 Quantifiers Instantiations
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Cumulative number of instantiations
 
 
Time (line number)
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0
 
 
 
1000
 
 
 
2000
 
 
 
3000
 
 
 
4000
 
 
 
5000
 
 
 
6000
 
 
 
7000
 
 
 
 
0
 
 
 
50000
 
 
 
100000
 
 
 
150000
 
 
 
200000
 
 
 
250000
 
 
 
300000
 
 
 
350000
 
 
 
400000
 
 
<polyline fill="none" opacity="1" stroke="#FFE119" stroke-width="1" points="60,749 60,748 60,748 60,748 60,748 61,748 61,748 61,748 61,748 61,748 61,748 61,748 61,747 61,747 62,747 62,747 62,747 62,747 62,747 62,747 64,747 64,747 64,747 64,746 64,746 64,746 64,746 64,746 64,746 64,746 64,746 64,746 64,746 64,746 64,745 64,745 64,745 65,745 65,745 65,745 65,745 65,745 65,745 65,745 65,745 65,744 65,744 67,744 67,744 68,744 68,744 68,744 68,744 68,744 68,744 68,744 70,744 70,743 70,743 70,743 81,743 82,743 82,743 82,743 82,743 82,743 85,743 85,743 85,742 85,742 85,742 85,742 86,742 86,742 86,742 86,742 86,742 86,742 86,742 86,741 86,741 86,741 86,741 86,741 86,741 86,741 86,741 86,741 86,741 86,741 86,740 86,740 86,740 86,740 86,740 86,740 86,740 86,740 86,740 86,740 86,740 86,739 86,739 86,739 86,739 86,739 86,739 86,739 86,739 86,739 86,739 86,739 86,739 86,738 86,738 86,738 87,738 87,738 87,738 87,738 87,738 87,738 87,738 87,738 87,737 87,737 87,737 87,737 87,737 87,737 87,737 87,737 87,737 87,737 87,737 87,736 88,736 88,736 88,736 88,736 88,736 88,736 88,736 88,736 88,736 88,736 88,735 89,735 89,735 89,735 89,735 89,735 92,735 92,735 92,735 92,735 92,735 92,734 92,734 92,734 94,734 94,734 94,734 94,734 94,734 95,734 95,734 95,734 95,734 95,733 95,733 95,733 95,733 95,733 95,733 95,733 95,733 95,733 95,733 95,733 95,732 95,732 95,732 95,732 95,732 96,732 96,732 96,732 96,732 96,732 96,732 96,731 97,731 97,731 97,731 97,731 97,731 97,731 97,731 97,731 97,731 97,731 97,730 97,730 97,730 97,730 97,730 97,730 97,730 97,730 97,730 97,730 97,730 97,729 98,729 98,729 98,729 98,729 98,729 98,729 98,729 142,729 142,729 142,729 142,729 142,728 142,728 142,728 142,728 142,728 142,728 142,728 142,728 142,728 143,728 143,728 143,727 143,727 144,727 144,727 144,727 144,727 144,727 144,727 144,727 144,727 144,727 144,726 144,726 144,726 144,726 144,726 145,726 145,726 145,726 145,726 145,726 145,726 145,725 145,725 145,725 145,725 145,725 145,725 146,725 146,725 146,725 146,725 146,725 146,725 146,724 146,724 146,724 146,724 146,724 146,724 146,724 147,724 147,724 147,724 147,724 147,723 147,723 147,723 147,723 147,723 147,723 147,723 147,723 147,723 147,723 147,723 148,722 148,722 148,722 148,722 148,722 148,722 148,722 148,722 148,722 148,722 148,722 148,721 148,721 148,721 148,721 149,721 149,721 149,721 149,721 149,721 149,721 149,721 149,720 150,720 150,720 150,720 150,720 150,720 150,720 150,720 150,720 150,720 150,720 150,720 150,719 150,719 150,719 151,719 151,719 151,719 151,719 151,719 151,719 151,719 151,719 151,718 151,718 151,718 151,718 151,718 151,718 151,718 151,718 151,718 152,718 152,718 152,717 152,717 152,717 152,717 152,717 152,717 152,717 152,717 152,717 152,717 152,717 152,716 152,716 152,716 152,716 152,716 152,716 152,716 152,716 152,716 152,716 152,716 153,715 153,715 153,715 153,715 153,715 153,715 153,715 153,715 153,715 153,715 154,715 154,715 154,714 154,714 154,714 154,714 154,714 154,714 154,714 154,714 154,714 155,714 155,714 155,713 155,713 155,713 155,713 155,713 155,713 155,713 155,713 155,713 156,713 156,713 156,712 156,712 156,712 156,712 156,712 156,712 156,712 157,712 157,712 157,712 157,712 157,711 157,711 157,711 157,711 157,711 157,711 157,711 157,711 157,711 157,711 158,711 158,710 158,710 158,710 158,710 158,710 158,710 158,710 158,710 15

In [18]:
// Top instantiated quantified term.
model.id2s(&top.peek()?.1)

"basic"

In [19]:
// Final counter and `Ident` value for the top term.
top.peek()?

(7903, basic#)

In [20]:
// Move to the first user quantifier
while !model.term(&top.peek()?.1)?.name().unwrap_or("").starts_with("outputbpl") { top.pop(); }
top.peek()?

(331, #4429!9)

In [21]:
// Introspect a particular term, given by handle.
// Note that the version number `!9` was added during parsing for disambiguation and is not present in the original log file.
model.term(&Ident::from_str("#4429!9")?)?

Quant { name: "outputbpl.310:31", params: 2, triggers: [#284], body: #4443, var_names: Some([VarName { name: Symbol("v2"), sort: Symbol("T@$Value") }, VarName { name: Symbol("v1"), sort: Symbol("T@$Value") }]) }

In [22]:
model.term_data(&Ident::from_str("#4429!9")?)?.instantiations.len()

359

In [23]:
// Create a new chart only with quantifier instantiation inside the file `outputbpl`.
let user_instantiation_times = instantiation_times.clone().into_iter().filter(|(n, _)| n.starts_with("outputbpl")).collect::<Vec<_>>();

plot_instantiations(&user_instantiation_times, "Top 10 User Quantifiers Instantiations", 10)

Top 10 User Quantifiers Instantiations
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Cumulative number of instantiations
 
 
Time (line number)
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0
 
 
 
50
 
 
 
100
 
 
 
150
 
 
 
200
 
 
 
250
 
 
 
300
 
 
 
 
0
 
 
 
50000
 
 
 
100000
 
 
 
150000
 
 
 
200000
 
 
 
250000
 
 
 
300000
 
 
 
350000
 
 
 
400000
 
 
<polyline fill="none" opacity="1" stroke="#FFE119" stroke-width="1" points="255,749 256,746 257,744 291,742 292,740 293,738 296,736 296,734 297,731 301,729 302,727 305,725 307,723 308,721 308,719 309,717 341,714 341,712 342,710 346,708 347,706 347,704 348,702 351,700 352,697 386,695 386,693 387,691 391,689 394,687 394,685 394,682 395,680 396,678 398,676 398,674 398,672 399,670 400,668 400,665 401,663 402,661 403,659 403,657 404,655 404,653 405,651 405,648 406,646 406,644 448,642 448,640 448,638 449,636 449,633 450,631 450,629 451,627 451,625 452,623 452,621 452,619 452,616 452,614 452,612 452,610 452,608 452,606 452,604 452,602 452,599 452,597 453,595 454,593 454,591 455,589 455,587 456,584 456,582 456,580 456,578 456,576 456,574 456,572 456,570 456,567 456,565 456,563 456,561 457,559 457,557 457,555 458,553 460,550 460,548 461,546 461,544 462,542 462,540 480,538 481,536 481,533 482,531 482,529 482,527 483,525 483,523 483,521 483,518 483,516 483,514 483,512 483,510 490,508 490,506 491,504 511,501 512,499 513,497 514,495 516,493 517,491 533,489 534,487 535,484 536,482 536,480 547,478 548,476 551,474 551,472 554,469 554,467 556,465 557,463 557,461 557,459 557,457 569,455 573,452 574,450 574,448 584,446 584,444 585,442 585,440 586,438 586,435 586,433 587,431 587,429 594,427 594,425 612,423 613,420 615,418 617,416 617,414 618,412 619,410 620,408 620,406 624,403 626,401 626,399 627,397 632,395 632,393 633,391 633,389 635,386 635,384 638,382 639,380 652,378 653,376 655,374 655,372 656,369 656,367 657,365 657,363 658,361 658,359 658,357 658,354 658,352 658,350 659,348 660,346 660,344 660,342 661,340 661,337 661,335 664,333 664,331 665,329 665,327 666,325 666,323 666,320 667,318 668,316 668,314 669,312 669,310 669,308 669,305 686,303 687,301 687,299 688,297 688,295 688,293 688,291 688,288 688,286 688,284 688,282 688,280 689,278 689,276 689,274 689,271 689,269 689,267 689,265 694,263 694,261 694,259 695,256 695,254 695,252 712,250 713,248 714,246 715,244 715,242 716,239 719,237 723,235 723,233 724,231 725,229 725,227 726,225 726,222 728,220 728,218 729,216 729,214 741,212 742,210 744,208 744,205 744,203 744,201 745,199 745,197 760,195 760,193 762,190 762,188 763,186 763,184 772,182 773,180 779,178 779,176 780,173 780,171 789,169 789,167 795,165 796,163 796,161 797,159 797,156 798,154 800,152 800,150 801,148 802,146 819,144 820,141 820,139 820,137 822,135 822,133 842,131 842,129 843,127 847,124 849,122 849,120 850,118 850,116 871,114 871,112 872,110 872,107 872,105 873,103 873,101 894,99 898,97 902,95 902,92 903,90 921,88 921,86 922,84 922,82 923,80 924,78 925,75 926,73 942,71 950,69 951,67 951,65 952,63 967,61 970,58 970,56 971,54 971,52 972,50 984,48 984,46 "/>
<polyline fill="none" opacity="1" stroke="#0082C8" stroke-width="1" points="255,749 256,746 256,744 290,742 291,740 292,738 295,736 296,734 297,731 301,729 305,727 307,725 307,723 308,721 309,719 340,717 341,714 341,712 346,710 346,708 347,706 347,704 350,702 351,700 352,697 385,695 386,693 387,691 389,689 391,687 393,685 394,682 394,680 394,678 395,676 397,674 398,672 398,670 398,668 399,665 400,663 400,661 400,659 401,657 402,655 403,653 403,651 404,648 404,646 404,644 406,642 447,640 448,638 448,636 449,633 450,631 450,629 451,627 451,625 452,623 452,621 452,619 452,616 452,614 452,612 452,610 452,608 452,606 452,604 452,602 452,599 452,597 452,595 453,593 